In [67]:
import numpy as np
import json

import sys
sys.path.append('../PlotNeuralNet-master/')
from pycore.tikzeng import *

# Classes

In [36]:
# Convolution
class Convolution:
    def __init__(self, kernel, padding, stride, nb_filter, fct_activation):
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [37]:
# input
class InputLayer:
    def __init__(self, shape):
        self.shape = shape

In [38]:
# Pooling Avg/Max
class Pooling:
    def __init__(self, op, kernel=2, padding="valid", stride=None):
        self.op = op # is the operation wanted (avg/max)
        self.kernel = kernel
        self.padding = padding
        self.stride = stride

In [39]:
# Class Flatten
class Flatten:
    pass

In [40]:
# Dense --> Fully connected layer
class Dense:
    def __init__(self, nb_neurones, fct_activation):
        self.nb_neurones = nb_neurones
        self.fct_activation = fct_activation

In [41]:
# identity block 
class IdBlock:
    def __init__(self, kernel, padding, nb_filter):
        self.kernel = kernel
        self.padding = padding
        self.nb_filter = nb_filter
        self.stride = 1

In [42]:
# convolution block
class ConvBlock:
    def __init__(self, kernel, padding, nb_filter, stride):
        self.kernel = kernel
        self.nb_filter = nb_filter
        self.padding = padding
        self.stride = stride

# Functions

In [43]:
# read json file 
# return list of layer + graph
def readJsonFile(json_file):
    with open(json_file) as archi_json:
        file = json.load(archi_json)
    
    
    # instantiate class
    list_layer = []
    for layer in file:
        parameters = layer["parameters"] # get parameters
        # if is Input class
        if(layer["class"] == "InputLayer"):
            # instantiate class and add into the list
            list_layer.append(InputLayer(parameters["shape"]))

        
        # if is Pooling class
        elif(layer["class"] == "Pooling"):
            try:
                padding = int(parameters["padding"])
            except:
                padding = "'" + parameters["padding"] + "'"
                
            list_layer.append(Pooling(parameters["op"],
                                      parameters["kernel"],
                                      padding,
                                      parameters["stride"]))
            
        # if is Convolution class
        elif(layer["class"] == "Convolution"):
            try:
                padding = int(parameters["padding"])
            except:
                padding = "'" + parameters["padding"] + "'"
                
                
            list_layer.append(Convolution(parameters["kernel"],
                                          padding,
                                          parameters["stride"],
                                          parameters["nb_filter"],
                                          parameters["fct_activation"]))
            
        # if is Flatten class
        elif(layer["class"] == "Flatten"):
            list_layer.append(Flatten())
        
        # if is Dense class
        elif(layer["class"] == "Dense"):
            list_layer.append(Dense(parameters["nb_neurones"],
                                    parameters["fct_activation"]))
        
        elif(layer["class"] == "IdBlock"):
            list_layer.append(IdBlock(parameters["kernel"],
                                    parameters["padding"],
                                    parameters["nb_filter"]))
            
        elif(layer["class"] == "ConvBlock"):
            list_layer.append(ConvBlock(parameters["kernel"],
                                        parameters["padding"],
                                        parameters["nb_filter"],
                                        parameters["stride"]))

        else : print("Error")
    return list_layer


In [44]:
# return import of the py file
def writeImport():
    return """
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
import sys
import traceback
import csv
from time import time
"""

In [45]:
# write mnist data set
def writeMnistDataset():
    return """(train_x, train_y), (test_x, test_y) = keras.datasets.mnist.load_data()

# normaliser les pixel 0-255 -> 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = tf.expand_dims(train_x, 3)
test_x = tf.expand_dims(test_x, 3)

val_x = train_x[:5000]
val_y = train_y[:5000]

"""

In [46]:
# write cifar data set
def writecifar10Dataset():
    return """
# load dataset
(train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()

# normalize to range 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

val_x = train_x[:5000]
val_y = train_y[:5000]
    
"""

In [47]:
# write function for identity block
def write_identity_block():
    return """
    def id_block(X, f, filters):
   
        X_shortcut = X

        X = Conv2D(filters=filters, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
        # X = BatchNormalization(axis=3)(X)
        X = Activation('relu')(X)


        X = Conv2D(filters=filters, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
        # X = BatchNormalization(axis=3)(X)

        X = Add()([X, X_shortcut])# SKIP Connection
        X = Activation('relu')(X)

        return X
    """

In [48]:
def write_conv_block():
    return"""
    def conv_block(X, f, filters, s=2):
    
        X_shortcut = X

        X = Conv2D(filters=filters, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
        # X = BatchNormalization(axis=3)(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=filters, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
        # X = BatchNormalization(axis=3)(X)

        X_shortcut = Conv2D(filters=filters, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
        # X_shortcut = BatchNormalization(axis=3)(X_shortcut)


        X = Add()([X, X_shortcut])
        X = Activation('relu')(X)

        return X
    """

In [49]:
# return string of the wanted layer
def write_layer(layer):
    
    # if is Intput
    if isinstance(layer, InputLayer):
        return "        X_input = X = Input({})\n".format(
                layer.shape)
        
            
    # if is Convolution
    elif isinstance(layer, Convolution):
        return "        X = Conv2D({}, kernel_size={}, strides={}, activation='{}', padding={})(X)\n".format(
                layer.nb_filter,
                layer.kernel,
                layer.stride,
                layer.fct_activation,
                layer.padding)
            
    # if is Pooling
    elif isinstance(layer, Pooling):
        if(layer.op == "avg"): # avg Pooling 
            return "        X = AveragePooling2D(pool_size={}, strides={}, padding={})(X)\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
        else : # Max Pooling
            return  "        X = MaxPooling2D(pool_size={}, strides={}, padding={})(X)\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
    elif isinstance(layer, IdBlock):
        return "        X = id_block(X, {}, {})\n".format(layer.kernel, layer.nb_filter)
    elif isinstance(layer, ConvBlock):
        return "        X = conv_block(X, {}, {}, {})\n".format(layer.kernel, layer.nb_filter, layer.stride)
    
    # Not possible
    else : print("Not Possible")

In [50]:
def end_CNN(layer):
    # if is Dense (aka Fully connected layer)
    if isinstance(layer, Dense):
        return "    head_model = Dense({}, activation='{}')(head_model)\n".format(
                        layer.nb_neurones, 
                        layer.fct_activation)
    # if is flatten
    elif isinstance(layer, Flatten):
        return "    head_model = Flatten()(head_model)\n"
    # Not possible
    else : print("Not Possible")

In [51]:
def create_CNN(archi):
    
    i = 0
    while not (isinstance(archi[i], Flatten)):
        i+=1

    end_archi = archi[i:]
    
    str_model_cnn = "def ResNet():\n"
    
    
    for layer in archi[:i]:
        str_model_cnn += write_layer(layer)
    
        
    str_model_cnn += "        model = Model(inputs=X_input, outputs=X, name='ResNet18')\n"
    str_model_cnn += "        return model\n\n"
    str_model_cnn += "    Input = ResNet()\n"
    str_model_cnn += "    head_model = Input.output\n"
    for end_layer in end_archi:
        str_model_cnn += end_CNN(end_layer)

    str_model_cnn += "    model = Model(inputs=Input.input, outputs=head_model)\n"
    return str_model_cnn

In [90]:
def create_py_file(json_file):
    s = json_file.split(".")
    file_name = s[0].split("/")[1]
    architecture = readJsonFile(json_file)
    
    CNN_tex(file_name, architecture)
    
    # python directory
    py_dir = "architecture_py/"
    
    # log directory
    log_dir = "../architecture_log/"
    
    # png directory
    png_dir = "../architecture_img/"
    
    # reset file
    file_py = open(py_dir + file_name + ".py", "w")
    file_py.close()
    
    file_py = open(py_dir + file_name + ".py", "a") # Open file in writting (a --> append)
    
    # write import
    file_py.write(writeImport())
    
    # write train/test data 
    file_py.write(writeMnistDataset())
    
    file_py.write("""

# init training time
training_time = 0
# init result test/train
test_result_loss = ""
test_result_acc = ""

train_result_loss = ""
train_result_acc = ""

nb_layers = "not build"

""")
    
    # try
    file_py.write("""try:
    """)
    
    # write function for id block
    file_py.write(write_identity_block())
    
    # write function for conv block
    file_py.write(write_conv_block())
    
    # write architecture model
    file_py.write(create_CNN(architecture))
    
    
    # write : create png of the model
    file_py.write("    plot_model(model, show_shapes=True, to_file=\"%s\")\n" % (png_dir+file_name+".png"))
    
    # write compiler
    file_py.write("""    model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])\n\n""")
    
    # write way for time computation
    file_py.write("""    start = time()\n""")
    
    # write model training
    file_py.write("""    model.fit(train_x, train_y, epochs=5, validation_data=(val_x, val_y))\n""")
    
    # write time computation
    file_py.write("""    training_time = time()-start\n""")
    
    # write model evaluation
    file_py.write("""    print(model.evaluate(test_x, test_y))\n""")
    
    
    # all is great
    log_file = log_dir + file_name +".log"
    file_py.write("""
    log_file = open(\"""" + log_file + """\" , "w")
    
    # save test result
    log_file.write('test result : ' + str(model.evaluate(test_x, test_y)))
    test_result_loss = model.evaluate(test_x, test_y)[0]
    test_result_acc = model.evaluate(test_x, test_y)[1]
    
    # save train result
    log_file.write('train result : ' + str(model.evaluate(test_x, test_y)))
    train_result_loss = model.evaluate(train_x, train_y)[0]
    train_result_acc = model.evaluate(train_x, train_y)[1]
    
    print('OK: file """ + log_file +""" has been create')
    
    nb_layers = len(model.layers)
    log_file.close()
""")
    
    # something go wrong 
    error_file = log_dir + file_name + "_error.log"
    file_py.write("""except:
    print('error: file """ + error_file +""" has been create')
    error_file = open(\"""" + error_file + """\" , "w")
    traceback.print_exc(file=error_file)
    result_loss = "Error"
    result_acc = "Error"
    error_file.close()
""")
    
    file_py.write("""finally:
    file = open('../architecture_results_resnet.csv', 'a', newline ='')
    with file: 

        # identifying header   
        header = ['file_name', 'training_time(s)', 'test_result_loss', 'test_result_acc', 'train_result_acc', 'train_result_loss', 'nb_layers'] 
        writer = csv.DictWriter(file, fieldnames = header) 
      
        # writing data row-wise into the csv file 
        # writer.writeheader() 
        writer.writerow({'file_name' : '"""+ file_name + """',  
                         'training_time(s)': training_time,  
                         'test_result_loss': test_result_loss,
                         'test_result_acc': test_result_acc,
                         'train_result_acc': train_result_acc,
                         'train_result_loss': train_result_loss,
                         'nb_layers': nb_layers}) 
        print('add line into architecture_results.csv')
    file.close()
    """)
    
    
    
    
    # close
    file_py.close()

# Test

In [86]:
directory = "architecture_json/"
archi = readJsonFile(directory+"archi_resnet_test.json")

In [107]:
archi

# Create Py file

In [63]:
directory = "architecture_json/"
for i in range(1,11):
    file_name = "archi_resnet_"+str(i)+".json"
    print(directory+file_name)
    create_py_file(directory+file_name)
    print("")

architecture_json/archi_resnet_1.json

architecture_json/archi_resnet_2.json

architecture_json/archi_resnet_3.json

architecture_json/archi_resnet_4.json

architecture_json/archi_resnet_5.json

architecture_json/archi_resnet_6.json

architecture_json/archi_resnet_7.json

architecture_json/archi_resnet_8.json

architecture_json/archi_resnet_9.json

architecture_json/archi_resnet_10.json

